<a href="https://colab.research.google.com/github/asengardeon/imdb-sentimetal/blob/main/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instituição**: FURB  
**Curso**: Data Science  
**Disciplina**: Deep Learning  
**Acadêmico**: Leandro Vilson Battisti  
**Sobre**  
O objetivo deste trabalho é criar um modelo binário de aprendizado de máquina para classificação de textos. Para isso, será utilizado a base de dados IMDB que consiste de dados textuais de críticas positivas e negativas de filmes.

### Classificação de textos para análise de sentimentos

Base de dados 

Istruções:
- O objetivo deste trabalho é criar um modelo binário de aprendizado de máquina para classificação de textos. 
Para isso, será utilizado a base de dados [IMDb](http://ai.stanford.edu/~amaas/data/sentiment/), que consiste de dados textuais de críticas positivas e negativas de filmes
- Uma vez treinado, o modelo deve ter uma função `predict` que recebe uma string como parâmetro e retorna o valor 1 ou 0, aonde 1 significa uma crítica positiva e 0 uma crítica negativa
- O pré-processamento pode ser desenvolvidado conforme desejar (ex.: remoção de stopwords, word embedding, one-hot encoding, char encoding)
- É preferível que seja empregado um modelo de recorrência (ex.: rnn, lstm, gru) para a etapa de classificação
- Documente o código (explique sucintamente o que cada função faz, insira comentários em trechos de código relevantes)
- **Atenção**: Uma vez treinado o modelo final, salve-o no diretório do seu projeto e crie uma célula ao final do notebook contendo uma função de leitura deste arquivo, juntamente com a execução da função `predict`

Sugestões:
- Explorar a base de dados nas células iniciais do notebook para ter um melhor entendimento do problema, distribuição dos dados, etc
- Após desenvolver a estrutura de classificação, é indicado fazer uma busca de hiperparâmetros e comparar os resultados obtidos em diferentes situações

Prazo de entrega:
- 01-08-2021 às 23:59hs GMT-3

Formato preferível de entrega:
- Postar no portal Ava da disciplina o link do projeto no github (ou anexar o projeto diretamente no portal Ava)

luann.porfirio@gmail.com

In [20]:
!pip install torchtext

In [21]:
from torchtext import datasets
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch import nn
import torch
import time
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

In [22]:
train_iter, test_iter = datasets.IMDB()

In [23]:
#Definindo se roda em cpu ou gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
next(train_iter)


('neg',
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [25]:
#criação do tokenizador de texto e texto e criação do vocabularios base de avaliação do texto.
tokenizer = get_tokenizer('basic_english')
train_iter = datasets.IMDB(split='train')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [26]:
vocab(['here', 'is', 'an', 'example'])

[131, 9, 40, 464]

###Estes pipelines são utilizados para tokenizar uma linha do dataloader  e para definir um código numerico para o sentimento referente a linha

In [27]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 0 if x == 'neg' else 1

### Antes de enviar para o modelo, a função collate_fn funciona em um lote de amostras geradas a partir do DataLoader. A entrada para collate_fn é um lote de dados com o tamanho do lote no DataLoader e collate_fn os processa de acordo com os pipelines de processamento de dados declarados anteriormente

In [28]:

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [29]:
train_iter = datasets.IMDB(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

### Este é o modelo que irtemos utilizar. Um modelo de classificação de texto Linear

In [30]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        #utilizamo o Bag pois de forma resumida ele garante a estrutura necessária para textos de tamanhos diferentes além de melhorias de perforance
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

Construímos um modelo com a dimensão de de interna de 64. O tamanho do vocabulário é igual ao comprimento da instância do vocabulário. O número de classes é igual ao número de rótulos, neste caso sendo 2 pois é positivo ou negativo, mas a implementação este flexivel para rótulos novos no futuro

In [31]:
train_iter = datasets.IMDB(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [32]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()        
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

In [33]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [34]:
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None


In [35]:
train_iter, test_iter = datasets.IMDB()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

In [36]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  4.34s | valid accuracy    0.709 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  4.28s | valid accuracy    0.793 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  4.26s | valid accuracy    0.819 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  4.01s | valid accuracy    0.711 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  4.04s | valid accuracy    0.858 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  3.98s |

In [37]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.862


In [41]:
ag_news_label = {0: "Negative",
                 1: "Positive"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "A frantic and cold-to-the-touch film of anxious set pieces and paper-thin characters, “Jolt” tracks the misadventures of the abovementioned New Yorker, an unpredictably sturdy woman cursed with a rare neurological disorder that makes it impossible for her to manage her anger and control her violent impulses triggered by annoying strangers. Truth be told, some folks throughout the film—from frustrating man-spreaders on the subway to arrogant bigwigs rude to service staff—test one’s patience, and perhaps even deserve the kind bloody rage Lindy feels the urge to disseminate at various hours of any given day. But in order to function in a society and city where rudeness is often the norm or background noise at best, Lindy has no choice but indulge in the painful experimental treatment her mysterious, somewhat patronizing psychiatrist Dr. Munchin (Stanley Tucci) seems to have invented"

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Positive news
